In [ ]:
# Prediction of master/key transcription factors based on following feature scores:
# Transcriptome based EXP (log2 normalized TPM) and JSD (tissue specificity) scores
# Epigenome based SERA (super enhancer regulatory activity) and MotifES (Motif Enrichment Score) scores
# These feature scores can be generated with helper_Transcriptome_EXP_JSD_scores.ipynb and 
# helper_Epigenome_SERA_MotifES_scores.ipynb

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import norm

In [ ]:
import pymc as pm
# check pymc version
print(f"Running on PyMC version {pm.__version__}")
import pytensor
import pytensor.tensor as at

In [ ]:
# load query's feature scores
# generated with helper_Transcriptome_EXP_JSD_scores.ipynb and helper_Epigenome_SERA_MotifES_scores.ipynb
query_tf_scores_df=pd.read_csv("query_combinedScores_SERA_MotifES_EXP_JSD.txt",sep="\t")
query_tf_scores_df.head()

In [ ]:
# get all Transcription factors' symbol
TF_symbol_list=query_tf_scores_df["Symbol"]

In [ ]:
# get query's feature scores
SERA_data=np.array(query_tf_scores_df["query_SERA"])
MotifES_data=np.array(query_tf_scores_df["query_MotifES"])
EXP_data=np.array(query_tf_scores_df["query_EXP"])
JSD_data=np.array(query_tf_scores_df["query_JSD"])

In [ ]:
# check histogram of data
plt.hist(JSD_data)

In [ ]:
# setup MTFinder model
with pm.Model() as mtf_model:
    # this is the fraction that come from master TF vs normal TF,
    # informative prior that about 10-20 master TFs in each cell/tissue
    p = pm.Beta( "p", alpha=2 , beta=300)
    # ensure p >= 0.001
    p_min_potential = pm.Potential("p_min_potential", at.switch(p < 0.001, -np.inf, 0))
    cluster = pm.Bernoulli( "ber", p = p, shape=len(TF_symbol_list))

    # precision with uniform prior
    SERA_sigma = pm.Uniform('SERA_sigma', lower=0.0, upper=np.std(SERA_data)*3, shape=2)
    MotifES_sigma = pm.Uniform('MotifES_sigma', lower=0.0, upper=np.std(MotifES_data)*3, shape=2)
    EXP_sigma = pm.Uniform('EXP_sigma', lower=0.0, upper=np.std(EXP_data)*3, shape=2)
    JSD_sigma = pm.Uniform('JSD_sigma', lower=0.0, upper=np.std(JSD_data)*3, shape=2)
    
    SERA_tau=pm.Deterministic('SERA_tau', SERA_sigma[cluster]**-2)
    MotifES_tau=pm.Deterministic('MotifES_tau', MotifES_sigma[cluster]**-2)
    EXP_tau=pm.Deterministic('EXP_tau', EXP_sigma[cluster]**-2)
    JSD_tau=pm.Deterministic('JSD_tau', JSD_sigma[cluster]**-2)
    
    # mean with uniform prior
    #SERA_mean=pm.Uniform("SERA_mean", lower=min(SERA_data), upper=max(SERA_data), shape=2)
    #MotifES_mean=pm.Uniform("MotifES_mean", lower=min(MotifES_data), upper=max(MotifES_data), shape=2)
    #EXP_mean=pm.Uniform("EXP_mean", lower=min(EXP_data), upper=max(EXP_data), shape=2)
    #JSD_mean=pm.Uniform("JSD_mean", lower=min(JSD_data), upper=max(JSD_data), shape=2)
    
    # mean with normal prior
    SERA_mean=pm.Normal("SERA_mean", mu=np.mean(SERA_data), sigma=np.std(SERA_data)*2, shape=2)
    MotifES_mean=pm.Normal("MotifES_mean", mu=np.mean(MotifES_data), sigma=np.std(MotifES_data)*2, shape=2)
    EXP_mean=pm.Normal("EXP_mean", mu=np.mean(EXP_data), sigma=np.std(EXP_data)*2, shape=2)
    JSD_mean=pm.Normal("JSD_mean", mu=np.mean(JSD_data), sigma=np.std(JSD_data)*2, shape=2)
    
    # break symmetry
    order_SERA_mean_potential = pm.Potential( "order_SERA_mean_potential", 
                                                at.switch(SERA_mean[1] - SERA_mean[0] < 0, -np.inf, 0) )
    order_MotifES_mean_potential = pm.Potential( "order_MotifES_mean_potential", 
                                                at.switch(MotifES_mean[1] - MotifES_mean[0] < 0, -np.inf, 0) )
    order_EXP_mean_potential = pm.Potential( "order_EXP_mean_potential", 
                                                at.switch(EXP_mean[1] - EXP_mean[0] < 0, -np.inf, 0) )
    order_JSD_mean_potential = pm.Potential( "order_JSD_mean_potential", 
                                                at.switch(JSD_mean[1] - JSD_mean[0] < 0, -np.inf, 0) )
    
    SERA_mu = pm.Deterministic('SERA_mu', SERA_mean[cluster])
    MotifES_mu = pm.Deterministic('MotifES_mu', MotifES_mean[cluster])
    EXP_mu = pm.Deterministic('EXP_mu', EXP_mean[cluster])
    JSD_mu = pm.Deterministic('JSD_mu', JSD_mean[cluster])
    
    # observed data
    SERA_process = pm.Normal('SERA_process', mu=SERA_mu, tau=SERA_tau, observed=SERA_data)
    MotifES_process = pm.Normal('MotifES_process', mu=MotifES_mu, tau=MotifES_tau, observed=MotifES_data)
    EXP_process = pm.Normal('EXP_process', mu=EXP_mu, tau=EXP_tau, observed=EXP_data)
    JSD_process = pm.Normal('JSD_process', mu=JSD_mu, tau=JSD_tau, observed=JSD_data)
    

In [ ]:
# run MTFinder model
with mtf_model:
    # initialize in a reasonable place to reduce the number of steps required
    start=pm.find_MAP()
    start["p"] = 0.01
    # sigma
    start["SERA_sigma"] = np.array([np.std(SERA_data), np.std(SERA_data)])
    start["MotifES_sigma"] = np.array([np.std(MotifES_data), np.std(MotifES_data)])
    start["EXP_sigma"] = np.array([np.std(EXP_data), np.std(EXP_data)])
    start["JSD_sigma"] = np.array([np.std(JSD_data), np.std(JSD_data)])
    # mean
    start["SERA_mean"] = np.array([np.mean(SERA_data), np.mean(SERA_data) + np.std(SERA_data)])
    start["MotifES_mean"] = np.array([np.mean(MotifES_data), np.mean(MotifES_data) + np.std(MotifES_data)])
    start["EXP_mean"] = np.array([np.mean(EXP_data), np.mean(EXP_data) + np.std(EXP_data)])
    start["JSD_mu"] = np.array([np.mean(JSD_data), np.mean(JSD_data) + np.std(JSD_data)])
    # MCMC sampling
    step1=pm.NUTS()
    step2=pm.BinaryMetropolis([cluster],scaling=0.01)
    idata=pm.sample(20000,start=start,step=[step1,step2]) # adjust steps as needed

In [ ]:
# check trace plot
#az.plot_trace(idata)
#az.plot_trace(idata, var_names=["SERA_mean"])
#az.plot_trace(idata, var_names=["SERA_sigma"])

In [ ]:
# check variable summary statics
az.summary(idata, var_names=["SERA_mean"])

In [ ]:
# calculate log odd ratio based on MCMC result
def calLogRatio(data_list, idata, var_mean, var_sigma ):
    """ Given two normal distribution, calculate the likehood ratio of a dataPoint belong to which normal distribution
    """
    mean1,mean2=az.summary(idata, var_names=[var_mean])["mean"]
    sigma1,sigma2=az.summary(idata, var_names=[var_sigma])["mean"]
    logRatio_list=[(norm.logpdf(datapoint, loc=mean2, scale=sigma2)-norm.logpdf(datapoint, loc=mean1, scale=sigma1)) for datapoint in data_list]
    return logRatio_list

In [ ]:
SERA_logRatio_list=calLogRatio(SERA_data, idata, "SERA_mean", "SERA_sigma" )
MotifES_logRatio_list=calLogRatio(MotifES_data, idata, "MotifES_mean", "MotifES_sigma" )
EXP_logRatio_list=calLogRatio(EXP_data, idata, "EXP_mean", "EXP_sigma" )
JSD_logRatio_list=calLogRatio(JSD_data, idata, "JSD_mean", "JSD_sigma" )
logRatio_dict={"TF_Symbol": TF_symbol_list, "SERA_logRatio": SERA_logRatio_list, "MotifES_logRatio":MotifES_logRatio_list, \
               "EXP_logRatio": EXP_logRatio_list, "JSD_logRatio": JSD_logRatio_list}
logRatio_df=pd.DataFrame(logRatio_dict)
logRatio_df["combined"] = logRatio_df["SERA_logRatio"] + logRatio_df["MotifES_logRatio"] +\
                                        logRatio_df["EXP_logRatio"] + logRatio_df["JSD_logRatio"]

In [ ]:
# sort based on combined score
logRatio_df=logRatio_df.sort_values('combined',ascending=False)
logRatio_df=logRatio_df.reset_index(drop=True)
logRatio_df.head()

In [ ]:
# get top 50 predicted MTF candidates
MTF_top50=logRatio_df.loc[:50,"TF_Symbol"]
# save result to file  
MTF_top50.to_csv('query_combinedlog2Scores_result_top50_MTF_candidates.txt',sep="\t")